In [1]:
import codecs, csv, sqlite3
import pandas as pd
import spacy
nlp = spacy.load('en')

In [54]:
conn = sqlite3.connect("output/lyrics.db")

cur = conn.cursor()
res = cur.execute(
    '''SELECT artist, album, genre, lyrics
    FROM lyrics
    WHERE genre in ('Progressive Metal','Thrash Metal','Death Metal')
    LIMIT 10''')
res = [line for line in res]
    
conn.close()

### Cleaner

In [55]:
suffixes = [("'ll "," will "),("'m "," am "),("'re "," are "),("'ve ", ' have '),("'d ", ' would '),("n't ", ' not '),
           (" he's "," he is "),(" she's "," she is "),(" it's "," it is ")]
suffixes = suffixes + [("`ll "," will "),("`m "," am "),("`re "," are "),("`ve ", ' have '),("`d ", ' would '),("n`t ", ' not '),
           (" he`s "," he is "),(" she`s "," she is "),(" it`s "," it is ")]
res[0][2]

'"I am the unclean The black drop at the bottom of your cup You\'d better drink or throw me up \'Cause I am on your lip and tongue God I\'m not yours as much as you are mine So let me in to be your lung Just breathe me deep and take another sip So still A taste so sweet but so bitter the kill Still on your lip You are so close I\'ll let you come Between my legs you are closer death than sun And I\'m not your daughter as much as you\'re my son I\'ll let you come In my mouth on your lip So ready and thirsty for the next sip You let me in, I let you come I\'d never let you down You let me win, I let you drown!" Getting used to pain  I am crying unwept tears through this violence I\'ll die trying to break this thick crust of silence  "I am the greatest star So bright that you all come forth and beg to taste my light I can take you far but I\'ll burn you out before we get there But hey! Who am I to stand in your way? Go ahead; swallow me down! I\'ll have no problems finding myself out When 

In [59]:
suffixes

[("'ll ", ' will '),
 ("'m ", ' am '),
 ("'re ", ' are '),
 ("'ve ", ' have '),
 ("'d ", ' would '),
 ("n't ", ' not '),
 (" he's ", ' he is '),
 (" she's ", ' she is '),
 (" it's ", ' it is '),
 ('`ll ', ' will '),
 ('`m ', ' am '),
 ('`re ', ' are '),
 ('`ve ', ' have '),
 ('`d ', ' would '),
 ('n`t ', ' not '),
 (' he`s ', ' he is '),
 (' she`s ', ' she is '),
 (' it`s ', ' it is ')]

In [56]:
import re
def cleaner(text):
    for suffix in suffixes:
        text = text.replace(suffix[0],suffix[1])
        return(text)

In [57]:
res = [(song[0:2], song[1], nlp(cleaner(song[2]))) for song in res]

In [58]:
for song in res:
    for token in song[2]:
        if token.pos_ in ('NOUN','ADJ','ADV'):
            print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_)


unclean unclean ADJ JJ amod
black black ADJ JJ amod
drop drop NOUN NN attr
bottom bottom NOUN NN pobj
your -PRON- ADJ PRP$ poss
cup cup NOUN NN pobj
better better ADV RB advmod
your -PRON- ADJ PRP$ poss
lip lip NOUN NN pobj
tongue tongue NOUN NN conj
not not ADV RB neg
yours yours ADJ PRP$ attr
as as ADV RB advmod
much much ADV RB advmod
So so ADV RB advmod
your -PRON- ADJ PRP$ poss
lung lung NOUN NN attr
Just just ADV RB advmod
deep deep ADV RB advmod
sip sip NOUN NN dobj
So so ADV RB advmod
still still ADV RB advmod
taste taste NOUN NN ROOT
so so ADV RB advmod
sweet sweet ADJ JJ amod
so so ADV RB advmod
bitter bitter ADJ JJ conj
kill kill NOUN NN dobj
Still still ADV RB advmod
your -PRON- ADJ PRP$ poss
lip lip NOUN NN pobj
so so ADV RB advmod
close close ADJ JJ acomp
my -PRON- ADJ PRP$ poss
legs leg NOUN NNS pobj
closer close ADJ JJR amod
death death NOUN NN attr
sun sun NOUN NN pobj
not not ADV RB neg
your -PRON- ADJ PRP$ poss
daughter daughter NOUN NN attr
as as ADV RB advmod
much 

When when ADV WRB advmod
morning morning NOUN NN compound
sheets sheet NOUN NNS pobj
tears tear NOUN NNS attr
your -PRON- ADJ PRP$ poss
mouth mouth NOUN NN pobj
weight weight NOUN NN attr
your -PRON- ADJ PRP$ poss
shoulder shoulder NOUN NN pobj
scream scream NOUN NN attr
that that ADJ WDT nsubj
my -PRON- ADJ PRP$ poss
heart heart NOUN NN nsubj
just just ADV RB advmod
n't not ADV RB neg
colder cold ADJ JJR acomp
Now now ADV RB advmod
rusty rusty ADJ JJ amod
heart heart NOUN NN nsubj
my -PRON- ADJ PRP$ poss
gift gift NOUN NN attr
love love NOUN NN nsubj
my -PRON- ADJ PRP$ poss
rift rift NOUN NN nmod
Earth earth NOUN NN pobj
never never ADV RB neg
life life NOUN NN dobj
that that ADJ WDT dobj
never never ADV RB neg
really really ADV RB advmod
way way NOUN NN nsubj
beauty beauty NOUN NN nsubj
eye eye NOUN NN dobj
Much much ADV RB advmod
more more ADJ JJR advmod
filth filth NOUN NN pobj
pain pain NOUN NN conj
ever ever ADV RB advmod
tears tear NOUN NNS attr
your -PRON- ADJ PRP$ poss
mouth m

true true ADJ JJ acomp


In [88]:
def filter_tags(songs, tags=('NOUN','ADJ','ADV')):
    filteredsongs = []
    for song in songs:
        song = (song[0], song[1], nlp(cleaner(song[2])))
        song = [token for token in song if token.pos_ in tags]
        filteredsongs.append(song)
    return(filteredsongs)

### Doc to Vec 

In [2]:
import spacy
import codecs, csv, sqlite3, re
nlp = spacy.load('en')

conn = sqlite3.connect("output/lyrics.db")

cur = conn.cursor()
res = cur.execute(
    '''SELECT artist, album, genre, lyrics
    FROM lyrics
    WHERE genre in ('Progressive Metal','Thrash Metal','Death Metal')
    ''')
res = [line for line in res]
    
conn.close()

suffixes = [("'ll "," will "),("'m "," am "),("'re "," are "),("'ve ", ' have '),("'d ", ' would '),("n't ", ' not '),
           (" he's "," he is "),(" she's "," she is "),(" it's "," it is ")]
suffixes = suffixes + [("`ll "," will "),("`m "," am "),("`re "," are "),("`ve ", ' have '),("`d ", ' would '),("n`t ", ' not '),
           (" he`s "," he is "),(" she`s "," she is "),(" it`s "," it is ")]

def cleaner(text):
    for suffix in suffixes:
        text = re.sub(suffix[0],suffix[1],text)
    text = text.lower()
    return(text)

In [3]:
import requests
resp = requests.get("https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords")
sws  = resp.text.split('\n')

In [4]:
with open('output/sws_en.csv', 'w', newline='') as sws_en:
    for word in sws:
        sws_en.write(word + '\n')

In [13]:
len(res)

3411

In [5]:
from collections import defaultdict

def freqdist(song, tags=('NOUN','ADJ','ADV')):
    fd = defaultdict(int)
    lyrics = nlp(cleaner(song[3]))
    for token in lyrics:
        if token.pos_ in tags and token.lemma_ not in sws + ['-PRON-']: fd[token.lemma_] += 1
    song = list(song) + [fd]
    return(song)

### All Progs

In [6]:
fds = []
vocab = []
counter = 1
for song in res:
    song = freqdist(song)
    fds.append(song[-1])
    vocab.extend(song[-1].keys())
    if counter % 50 == 0:print('%s is processed' % counter)
    counter += 1

vocab = set(vocab)

50 is processed
100 is processed
150 is processed
200 is processed
250 is processed
300 is processed
350 is processed
400 is processed
450 is processed
500 is processed
550 is processed
600 is processed
650 is processed
700 is processed
750 is processed
800 is processed
850 is processed
900 is processed
950 is processed
1000 is processed
1050 is processed
1100 is processed
1150 is processed
1200 is processed
1250 is processed
1300 is processed
1350 is processed
1400 is processed
1450 is processed
1500 is processed
1550 is processed
1600 is processed
1650 is processed
1700 is processed
1750 is processed
1800 is processed
1850 is processed
1900 is processed
1950 is processed
2000 is processed
2050 is processed
2100 is processed
2150 is processed
2200 is processed
2250 is processed
2300 is processed
2350 is processed
2400 is processed
2450 is processed
2500 is processed
2550 is processed
2600 is processed
2650 is processed
2700 is processed
2750 is processed
2800 is processed
2850 is proc

In [7]:
import csv    

with open('output/progtovec.csv', 'w', newline='') as progs:
    writer = csv.DictWriter(progs, fieldnames = ['artist_name','album_name','genre_name'] + list(vocab))
    writer.writeheader()
    for i,fd in enumerate(fds):
        fd['album_name'],fd['artist_name'],fd['genre_name'] = res[i][:3]
        writer.writerow(fd)

## Gensim

In [172]:
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary



In [ ]:

dct = Dictionary(dataset)  # fit dictionary
corpus = [dct.doc2bow(line) for line in dataset]  # convert dataset to BoW format

model = TfidfModel(corpus)  # fit model
vector = model[corpus[0]]  # apply model

In [173]:
dataset = api.load("text8")

[==================================================] 100.0% 31.6/31.6MB downloaded


In [177]:
dir(dataset)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'fn']